Data: https://www.kaggle.com/tunguz/russian-glove

In [ ]:
!pip install numpy nltk scikit-learn scipy pymorphy2

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords
from scipy.spatial import distance
import scipy
from pymorphy2 import MorphAnalyzer

In [ ]:
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print('Done.')
    return model

In [ ]:
def preprocess(raw_text):
    morph = MorphAnalyzer()
    find_cyr = re.sub('[^а-яА-Я]', ' ', raw_text)
    words = find_cyr.lower().split()
    stopword_set = set(stopwords.words("russian"))
    md_keys = set(model.keys())
    cleaned_words = list(set([w for w in words if w not in stopword_set]))
    contained = [word for word in cleaned_words if word in md_keys]
    return contained

In [ ]:
def cosine_distance_wordembedding_method(s1, s2):
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
    
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return round(((2-cosine)/2)*100,2)

In [ ]:
gloveFile = 'multilingual_embeddings.ru'
model = loadGloveModel(gloveFile)
sentences1 = ['Разработать программный пакет, моделирующий марковские процессы на графе  твердых разбиений',
'оценки геометрических параметров четырехмерных диаграмм Юнга в  марковских процессах:\
 псевдопланшерелевском процессе и процессе  Ричардсона',
'проверки гипотезы о слиянии жадных псевдопланшерелевских  последовательностей',
'исследования асимптотической центральности псевдопланшерелевского процесса.']

sentences2 = ['Разработан и реализован программный пакет,  моделирующий марковские процессы на графе твердых  разбиений',
'Получены оценки скорости роста длины поперечника и  количества угловых клеток',
'Компьютерные эксперименты показали, что 4D  псевдопланшерелевский процесс обладает свойством  асимптотической центральности',
'Не было выявлено ни одного контрпримера гипотезы о  слиянии жадных псевдопланшерелевских\
  последовательностей 4D диаграмм Юнга.'
]

for i in range(len(sentences1)):
    print("{} \n{} \nScore: {:.1f}\n".format(sentences1[i], sentences2[i], cosine_distance_wordembedding_method(sentences1[i], sentences2[i])))